In [10]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from stem import Signal
from stem.control import Controller
import time

# Function to request new Tor identity (new IP address)
def renew_tor_identity():
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM) 

# Configure Selenium to use Tor
def get_tor_firefox_browser():
    options = Options()
    options.headless = True  # Run in headless mode (no GUI)
    options.set_preference('network.proxy.type', 1)
    options.set_preference('network.proxy.socks', '127.0.0.1')
    options.set_preference('network.proxy.socks_port', 9050)
    options.set_preference('network.proxy.socks_remote_dns', True)  
    browser = webdriver.Firefox(options=options)
    return browser

In [11]:
# Initialize Firefox with Tor
#browser = setup_tor_firefox()
browser = get_tor_firefox_browser()

# Visit the website
url = 'http://lbb2llze7ab4rnq4jumsy4ihsqzpuysaofpz2e43foocwmrzsokumqid.onion/'  # Replace with your target URL
browser.get(url)
time.sleep(5)  # Allow time for page to load

In [12]:
# Scrape data (adjust as per the website structure)
#elements = browser.find_elements_by_xpath('//h1')  # Change based on what you need to scrape
companyNames = []
companyDescription = []
dateVictimised = []
noClicks = []
publishedStatus = []

cards_good = browser.find_elements(By.XPATH,"//a[@class='post-block good']")

# Loop through 'good' cards
for card in cards_good:
    dateVictimisedElement = card.find_element(By.XPATH,".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element(By.XPATH,".//div[@class='post-title']")
    #print(f"Scraping: {companyNameElement}")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element(By.XPATH,".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element(By.XPATH,".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    publishedStatus.append('Published')


cards_bad = browser.find_elements(By.XPATH,"//a[@class='post-block bad']")

# Loop through 'bad' cards
for card in cards_bad:
    dateVictimisedElement = card.find_element(By.XPATH,".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element(By.XPATH,".//div[@class='post-title']")
    #print(f"Scraping: {companyNameElement}")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element(By.XPATH,".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element(By.XPATH,".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    publishedStatus.append('Not Published')

#browser.quit()

# Store data
#data = [{'Title': element.text} for element in elements]

# Close the browser
#browser.quit()

# Save data into CSV
print("Scraping completed")

Scraping completed


In [13]:
print("Company Names:", companyNames)
print("Company Description:", companyDescription)
print("Date Victimised:", dateVictimised)
print("Number of Clicks:", noClicks)
print("Published Status:", publishedStatus)


Company Names: ['rhp.com.br', 'gbricambi.it', 'fanningfanning.com', 'akanea.com', 'jutebag.co.uk', 'designintoto.com.au', 'robesoncoso.org', 'acdcexpress.com', 'pkf.com.au', 'inces.com', 'huntongroup.com', 'hphood.com', 'bridgestoneamericas.com', 'albanybank.com', 'sampoernaagro.com', 'tdsb.on.ca', 'alconca.com.ve', 'policiaauxiliarcusaem.com.mx', 'ultragasmexico.com', 'taylorcrane.com', 'soken-ce.co.jp', 'sitrack.com', 'shinwajpn.co.jp', 'mogaisrael.com', 'maximumresearch.com', 'kioti.com', 'indoramaventures.com', 'hatsinteriors.com', 'grc-c.co.il', 'eif.org.na', 'coreengg.com', 'champion.com.co', 'centralepaysanne.lu', 'carlfischer.com', 'auburnpikapp.org', 'anwaltskanzlei-kaufbeuren.de', 'udhaiyamdhall.com', 'tetco.com', 'ipsenlogistics.com', 'inouemfg.com', 'ciasc.mx', 'cdtmedicus.pl', 'bildco.ae', 'vitema.vi.gov', 'vinatiorganics.com', 'vikrantsprings.com', 'unisi.it', 'acsistemas.com', 'thaiagri.com', 'snapethorpeprimary.co.uk', 'sislocar.com', 'ondozabal.com', 'museu-goeldi.br',

In [14]:
#browser.quit()

# Store data
#data = [{'Title': element.text} for element in elements]

# Close the browser
#browser.quit()

# Save data into CSV
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus})
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], format='Updated: %d %b, %Y, %H:%M UTC')
df = df.reset_index(drop=True)

df.to_csv('scraped_data.csv', index=False)

print("Scraping completed and saved to scraped_data.csv")

Scraping completed and saved to scraped_data.csv
